<a href="https://www.kaggle.com/code/mzaoualim/kaggle-icr?scriptVersionId=130456388" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Libraries imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#training 

train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv').set_index('Id')
## replacing missing values with zeros
train = train.fillna(0)
## converting categorical variables
train = train.replace({'EJ' : { 'A' : 0, 'B' : 1}})
## Scaling
train =pd.DataFrame(MinMaxScaler().fit_transform(train), columns=train.columns)
## y
y = train['Class']
## X
X = train.drop(columns='Class')
## Predict_Prob
SGDClassifier_model= SGDClassifier(loss='log_loss',
                                  penalty='l1', 
                                  alpha=0.0001, 
                                  l1_ratio=0.15, 
                                  fit_intercept=False, 
                                  max_iter=1000, 
                                  tol=0.001, 
                                  shuffle=True, 
                                  verbose=0, 
                                  epsilon=0.1, 
                                  n_jobs=-1, 
                                  random_state=88, 
                                  learning_rate='optimal', 
                                  eta0=0.0, 
                                  power_t=0.5, 
                                  early_stopping=False, 
                                  validation_fraction=0.30, 
                                  n_iter_no_change=10, 
                                  class_weight='balanced', # As our target var is imbalanced
                                  warm_start=False, 
                                  average=False)

SGDClassifier_model.fit(X, y)

SGDClassifier(class_weight='balanced', fit_intercept=False, loss='log_loss',
              n_iter_no_change=10, n_jobs=-1, penalty='l1', random_state=88,
              validation_fraction=0.3)

In [3]:
# Testing

test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv').set_index('Id')
## replacing missing values with zeros
test = test.fillna(0)
## converting categorical variables
test = test.replace({'EJ' : { 'A' : 0, 'B' : 1}})
## Scaling
test =pd.DataFrame(MinMaxScaler().fit_transform(test), columns=test.columns)
## Creating Submission_file
submission = pd.DataFrame(SGDClassifier_model.predict_proba(test), 
                          columns=['class_0','class_1']).round(4)
submission.insert(0,
                  'Id',
                  pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv',
                             usecols=['Id']))

submission.to_csv('submission.csv', index=False)